In [4]:
#0918:all question and column links from topic "crispr" 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup
from lxml import etree
from pyquery import PyQuery as pq
import pandas as pd
from pandas import Series, DataFrame
import csv
import urllib.request
import datetime
import time
import re

#------------------------------------------------------------1st stage,collect all question/column links
driver = webdriver.Chrome()
url = "https://www.zhihu.com/topic/20058152/hot"
    
driver.get(url)
driver.implicitly_wait(10)

time.sleep(2)

#scroll to the topic bottom
for i in range(70):
    js = "var q = document.documentElement.scrollTop="+str(i*1000)
    driver.execute_script(js)
    time.sleep(0.8)
time.sleep(2)

listql = []
soup = BeautifulSoup(driver.page_source,"lxml")
for b in soup.findAll("meta",attrs = {"itemprop":"url"}):
    c = b["content"]
    if "question" in c and "answer" not in c and c not in listql:
        listql.append(c) 
    if "zhuanlan" in c and c not in listql:
        listql.append(c) 

#url = "https://www.zhihu.com/topic/20741067/top-answers"
driver.find_element_by_link_text("精华").click()
time.sleep(2)
driver.implicitly_wait(10)
time.sleep(2)

#scroll to the topic bottom
for i in range(30):
    js = "var q = document.documentElement.scrollTop="+str(i*1000)
    driver.execute_script(js)
    time.sleep(0.8)
time.sleep(2)

soup = BeautifulSoup(driver.page_source,"lxml")
for b in soup.findAll("meta",attrs = {"itemprop":"url"}):
    c = b["content"]
    if "question" in c and "answer" not in c and c not in listql:
        listql.append(c) 
    if "zhuanlan" in c and c not in listql:
        listql.append(c) 

        
#url = "https://www.zhihu.com/topic/20741067/unanswered"
driver.find_element_by_link_text("等待回答").click()
driver.implicitly_wait(10)
time.sleep(2)

#scroll to the topic bottom
for i in range(25):
    js = "var q = document.documentElement.scrollTop="+str(i*1000)
    driver.execute_script(js)
    time.sleep(0.8)
time.sleep(2)

soup = BeautifulSoup(driver.page_source,"lxml")
        
for b in soup.findAll("a"):
    c = b["href"]
    d = "https://www.zhihu.com" + c
    if "question" in c and len(c) == 19 and "topic" not in c and "write_answer" not in c and d not in listql:
        listql.append(d)


listql = [re.sub("^https:/+","https://",i) for i in listql]

save = "/Users/CM/desktop/ScrapeZQ2/QClink/ql-crispr.csv"
df = pd.DataFrame(listql,columns = ["QClink"])
df1 = df.to_csv(save, index = False)
print(pd.read_csv(save))
#------------------------------------------------------------2nd stage,collect question/column details
driver.quit()

                                       QClink
0    https://www.zhihu.com/question/345628210
1       https://zhuanlan.zhihu.com/p/78802286
2       https://zhuanlan.zhihu.com/p/76296904
3       https://zhuanlan.zhihu.com/p/73374896
4       https://zhuanlan.zhihu.com/p/77583210
5       https://zhuanlan.zhihu.com/p/81630260
6       https://zhuanlan.zhihu.com/p/72092622
7       https://zhuanlan.zhihu.com/p/68993941
8       https://zhuanlan.zhihu.com/p/69043718
9    https://www.zhihu.com/question/270413798
10      https://zhuanlan.zhihu.com/p/68652031
11      https://zhuanlan.zhihu.com/p/81580033
12   https://www.zhihu.com/question/303911503
13    https://www.zhihu.com/question/22356523
14      https://zhuanlan.zhihu.com/p/50896679
15    https://www.zhihu.com/question/38103149
16   https://www.zhihu.com/question/303672438
17      https://zhuanlan.zhihu.com/p/29204199
18   https://www.zhihu.com/question/263722407
19      https://zhuanlan.zhihu.com/p/33204864
20   https://www.zhihu.com/questio

In [17]:
#0918:question and column detail from topic "crispr" 
driver = webdriver.Chrome()

#scan the QR code manually to login
driver.get("https://www.zhihu.com/")
time.sleep(12)

with open("/Users/CM/desktop/ScrapeZQ2/qca-crispr3.csv","w",encoding = "utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["column","title","flwr","v","authlink","upno","cmtno","andate","textlen","imgno","antext"])
    df1 = pd.read_csv("/Users/CM/desktop/ScrapeZQ2/QClink/qlerr-crispr2.csv")["QClink"].values

    for qcl in df1:
        driver.get(qcl)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source,"lxml")
    
        try:            
            if "question" in qcl:
                #unfold qtext
                time.sleep(1)
                try:
                    driver.find_element_by_xpath("//div[@class='QuestionRichText QuestionRichText--expandable QuestionRichText--collapsed']").click()
                    time.sleep(1)
                except:
                    time.sleep(1)
                
                #get question info
                column = "0"
                qtitle = soup.find("h1", class_="QuestionHeader-title").get_text()+soup.find("div", class_="QuestionHeader-detail").get_text()
                qflwr = soup.findAll("strong", class_="NumberBoard-itemValue")[0].get_text()
                qv = soup.findAll("strong", class_="NumberBoard-itemValue")[1].get_text()                

                if len(soup.findAll("h4", class_="List-headerText")) > 0:                   
                    #scroll to the bottom
                    xhd = soup.findAll("button", class_="Button QuestionAnswers-answerButton Button--blue Button--spread")
                    an = soup.findAll("span", class_="RichText ztext CopyrightRichText-richText")
                    if len(an) > 0:
                        n = 1
                        while len(xhd) == 0:
                            js = "var q = document.documentElement.scrollTop="+str(n*1000)
                            driver.execute_script(js)
                            soup = BeautifulSoup(driver.page_source,"lxml")
                            xhd = soup.findAll("button", class_="Button QuestionAnswers-answerButton Button--blue Button--spread")
                            n+=1
                            time.sleep(0.6)
                    
                    #get all answer info
                    ans = soup.findAll("div", class_="List-item")
                    for ai in ans:
                        try:
                            authlink = re.sub("^//","https://",ai.find("a", class_="UserLink-link")["href"])
                        except:
                            authlink = "AnonymousUser"
                        upno = ai.find("button", class_="Button VoteButton VoteButton--up").get_text()
                        cmtno = ai.find("button", class_="Button ContentItem-action Button--plain Button--withIcon Button--withLabel").get_text()
                        andate = ai.find("div", class_="ContentItem-time").get_text()
                        imgno = str(len(ai.findAll("figure")))+"个图片"
                        antext = ai.find("span", class_="RichText ztext CopyrightRichText-richText").get_text()
                        textlen = str(len(antext))+"个文字"
                        writer.writerow([column,qtitle,qflwr,qv,authlink,upno,cmtno,andate,textlen,imgno,antext])
                else:
                    authlink = "AnonymousUser"
                    writer.writerow([column,qtitle,qflwr,qv,authlink])                   
            #get column info        
            elif "zhuanlan" in qcl:
                 
                column = "1"
                coltitle = soup.find("h1", class_="Post-Title").get_text()
                colflwr = "NoColflwr"
                colv = "NoColv"
                authlink = re.sub("^//","https://",soup.find("a", class_="UserLink-link")["href"])
                upno = soup.find("button", class_="Button VoteButton VoteButton--up").get_text()
                cmtno = soup.find("button", class_="Button BottomActions-CommentBtn Button--plain Button--withIcon Button--withLabel").get_text()
                coldate = soup.find("div", class_="ContentItem-time").get_text()
                imgno = str(len(soup.findAll("figure")))+"个图片"
                coltext = soup.find("div", class_="Post-RichTextContainer").get_text()
                textlen = str(len(coltext))+"个文字"
                writer.writerow([column,coltitle,colflwr,colv,authlink,upno,cmtno,coldate,textlen,imgno,coltext])
            time.sleep(2)
            
        except:
            print(qcl) 
            #fill up manually

print("done.")
time.sleep(1)
driver.quit()


https://www.zhihu.com/question/345314210
https://www.zhihu.com/question/40392924
https://www.zhihu.com/question/328140295
https://www.zhihu.com/question/303748544
https://www.zhihu.com/question/60502266
https://www.zhihu.com/question/313198233
https://www.zhihu.com/question/64667047
https://www.zhihu.com/question/303824437
https://www.zhihu.com/question/266739756
https://www.zhihu.com/question/315251450
https://www.zhihu.com/question/311035305
https://www.zhihu.com/question/264687896
https://www.zhihu.com/question/303770302
https://www.zhihu.com/question/274037224
https://www.zhihu.com/question/317091956
https://www.zhihu.com/question/299495707
https://www.zhihu.com/question/300206829
https://www.zhihu.com/question/345906832
https://www.zhihu.com/question/280230894
https://www.zhihu.com/question/305818975
https://www.zhihu.com/question/305365605
https://www.zhihu.com/question/269686714
done.


In [47]:
a = "我认为贺健奎的实验目的不是增强人类的智力 (designer baby)因为从目前的研究结果来看, 有很多基因的单核苷酸多态性 (你可以认为是突变) 对智力的影响比CCR5大.(我随手翻了篇文章, 没看内容)而且很早之前就有'聪明鼠'的实验, 把一整个基因转到老鼠里, 老鼠变聪明了.所以一个聪明人如果想偷偷的做设计婴儿的实验, 有很多表型更明显的基因可以选择. 用人话表达就是他可以编辑其他基因, 能得到更强的婴儿.  CCR5即使对智力有影响, 恐怕效果也没有那么强.如果他的目的是弄出更聪明的婴儿, 那检测实验成功的难度就比较大了. 被编辑的儿童的智商总得超过人群平均值两个标准差, 也就是130以上, 才能宣称实验成功把..  这个风险太大了, 而且实验需要花费的时间也太长. 他恐怕收不回成本.CRISPR编辑基因会有很多意想不到的问题, 直接在人胚胎做毫无意义的编辑就更是瞎扯淡了....."
len(a)

401

In [64]:
#0918 question error page processing
qlerr = ["https://www.zhihu.com/question/269686714"]

driver = webdriver.Chrome()
#scan the QR code manually to login
#driver.get("https://www.zhihu.com/")
#time.sleep(8)

with open("/Users/CM/desktop/ScrapeZQ2/qca-crispr4.csv","w",encoding = "utf-8") as f:
        writer = csv.writer(f)
        for ql in qlerr:
            
            driver.get(ql)
            #time.sleep(8)
            soup = BeautifulSoup(driver.page_source,"lxml")
            
            try:
                
            
                #unfold qtext
                time.sleep(1)
                try:
                    driver.find_element_by_xpath("//div[@class='QuestionRichText QuestionRichText--expandable QuestionRichText--collapsed']").click()
                    time.sleep(1)
                except:
                    time.sleep(1)
                
                #get question info
                column = "0"
                qtitle = soup.find("h1", class_="QuestionHeader-title").get_text()+soup.find("div", class_="QuestionHeader-detail").get_text()
                qflwr = soup.findAll("strong", class_="NumberBoard-itemValue")[0].get_text()
                qv = soup.findAll("strong", class_="NumberBoard-itemValue")[1].get_text()                
    
                if len(soup.findAll("h4", class_="List-headerText")) > 0:                   
                    #scroll to the bottom
                    xhd = soup.findAll("button", class_="Button QuestionAnswers-answerButton Button--blue Button--spread")
                    an = soup.findAll("span", class_="RichText ztext CopyrightRichText-richText")
                    if len(an) > 0:
                        n = 1
                        while len(xhd) == 0:
                            js = "var q = document.documentElement.scrollTop="+str(n*1000)
                            driver.execute_script(js)
                            soup = BeautifulSoup(driver.page_source,"lxml")
                            xhd = soup.findAll("button", class_="Button QuestionAnswers-answerButton Button--blue Button--spread")
                            n+=1
                            time.sleep(0.6)
                    
                    #get all answer info
                    soup = BeautifulSoup(driver.page_source,"lxml")
                    ans = soup.findAll("div", class_="List-item")
                    for ai in ans:
                        try:
                            authlink = re.sub("^//","https://",ai.find("a", class_="UserLink-link")["href"])
                        except:
                            authlink = "AnonymousUser"
                        upno = ai.find("button", class_="Button VoteButton VoteButton--up").get_text()
                        cmtno = ai.find("button", class_="Button ContentItem-action Button--plain Button--withIcon Button--withLabel").get_text()
                        andate = ai.find("div", class_="ContentItem-time").get_text()
                        imgno = str(len(ai.findAll("figure")))+"个图片"
                        antext = ai.find("span", class_="RichText ztext CopyrightRichText-richText").get_text()
                        textlen = str(len(antext))+"个文字"
                        writer.writerow([column,qtitle,qflwr,qv,authlink,upno,cmtno,andate,textlen,imgno,antext])
                else:
                    authlink = "AnonymousUser"
                    writer.writerow([column,qtitle,qflwr,qv,authlink])  
            except:
                print(ql)


    
print("done.")
driver.quit()

done.


In [65]:
#0918:answerers detail in topic "基因驱动" 
driver = webdriver.Chrome()

#scan the QR code manually to login
driver.get("https://www.zhihu.com/")
time.sleep(10)

with open("/Users/CM/desktop/ScrapeZQ2/ai-crispr.csv","w",encoding = "utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["authlink","authname","verified","authflwr","authupno"])
    df2 = pd.read_csv("/Users/CM/desktop/ScrapeZQ2/qca-crispr.csv")["authlink"].values
    for al in df2:
        if al != "AnonymousUser":
            driver.get(al)
            driver.implicitly_wait(8)
            soup = BeautifulSoup(driver.page_source,"lxml")
            
            try:
                if len(soup.findAll("div", class_="ProfileMainPrivacy-mainContentWrapper")) == 0:
                    authlink = al
                    authname = soup.find("span",class_="ProfileHeader-name").get_text()
                    if soup.find("div", class_="Card-headerText").get_text() == "个人成就":
                        verified = "0"
                    if soup.find("div", class_="Card-headerText").get_text() == "认证与成就":
                        verified = "1"
                    authflwr = soup.findAll("strong", class_="NumberBoard-itemValue")[1].get_text()
                    for i in soup.findAll("div", class_="IconGraf"):
                        if "赞同" in i.get_text():
                            authupno = i.get_text()
                    writer.writerow([authlink,authname,verified,authflwr,authupno])
                    time.sleep(1)
            except:
                print(al)
                #fill up manually
print("done.")
            
time.sleep(1)
driver.quit()



https://www.zhihu.com/people/xi-ming-301
https://www.zhihu.com/people/yang-girl-65
https://www.zhihu.com/people/1ri-bian-shi-jiu-liao
https://www.zhihu.com/people/gerry-wong
https://www.zhihu.com/people/wo-xi-huan-80-54
https://www.zhihu.com/people/ji-sai-98
https://www.zhihu.com/people/chen-jing-lei-15
https://www.zhihu.com/people/shuki
https://www.zhihu.com/people/916c8ae2fe1fa95956f3c34ef1122ea0
https://www.zhihu.com/people/ban-ming-feng-59
https://www.zhihu.com/people/hanjingxue
https://www.zhihu.com/people/jing-cai-guang-yao
https://www.zhihu.com/people/bei-jing-jia-mei-shi-yan
https://www.zhihu.com/org/wang-sheng-sheng-wu-oneshine
https://www.zhihu.com/people/wu-ye-yang-guang-81-84
https://www.zhihu.com/people/xun-yuan-min
https://www.zhihu.com/people/wu-gan-lin-85
https://www.zhihu.com/people/long-gong-yu-41
https://www.zhihu.com/people/lilizvic
https://www.zhihu.com/people/ming-ren-jiu-tai-lang
done.
